In [3]:
"""
Vorlesung 2 – Datenvorbereitung: EIN kommentiertes Snippet
Themen (aus Teil A): CSV laden, Typen, Missingness, Imputation, Indikatoren, One-Hot, Skalierung
(Standardisierung/MinMax/Robust), Ausreißer-Kappen, Datumsfeatures, Duplikate, Train/Test, Pipeline.

Das Skript versucht zuerst, 'data.csv' aus dem Arbeitsverzeichnis zu laden.
Falls nicht vorhanden, wird ein kleines Beispiel-CSV aus einem String erzeugt.
"""

import io
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

# ------------------------------------------------------------
# 0) CSV LADEN (A8): robust mit na_values, Encoding, Fallback
# ------------------------------------------------------------
CSV_PATH = "data.csv"

NA_TOKENS = ["", " ", "NA", "N/A", "n/a", "NaN", "nan", "NULL", "null", "?", "-", "--"]

def load_csv_or_example(path):
    try:
        df = pd.read_csv(
            path,
            sep=",",               # passe an, falls ';'
            decimal=".",           # passe an, falls ','
            encoding="utf-8",
            na_values=NA_TOKENS,
            low_memory=False
        )
        source = f"geladen: {path}"
    except FileNotFoundError:
        demo = io.StringIO(
            """date,city,rooms,area_m2,price,has_elevator
            2025-01-03,Berlin,2,45,245000,yes
            2025-02-10,berlin,NA,60,315000,no
            2024-12-15,München,1,34,?,yes
            2025-01-20,Hamburg,3,85,520000,yes
            2025-03-02,München,2, ,480000,no
            2025-03-05,Berlin,2,55,310000,unknown
            """
        )
        df = pd.read_csv(demo, sep=",", na_values=NA_TOKENS)
        source = "Beispiel-CSV aus String"
    return df, source

df, src = load_csv_or_example(CSV_PATH)
print(f"[INFO] CSV {src}")
print("[INFO] Rohform:")
print(df.head(), "\n")

# ------------------------------------------------------------
# 1) SPALTENNAMEN & GRUNDBEREINIGUNG (A9)
# ------------------------------------------------------------
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_", regex=False)
)

# Strings trimmen, leere Strings -> NaN
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype(str).str.strip()
    df.loc[df[col].isin(["", "nan", "None", "none", "NaN"]), col] = np.nan

# Duplikate entfernen
before = len(df)
df = df.drop_duplicates()
print(f"[INFO] Duplikate entfernt: {before - len(df)}\n")

# ------------------------------------------------------------
# 2) DATUMS-/ZEITSPALTEN PARSEN (A7)
# ------------------------------------------------------------
for col in df.columns:
    if any(tok in col for tok in ["date", "time", "datum"]):
        df[col] = pd.to_datetime(df[col], errors="coerce")

date_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.datetime64)]
for col in date_cols:
    df[col + "_year"]  = df[col].dt.year
    df[col + "_month"] = df[col].dt.month
    df[col + "_dow"]   = df[col].dt.dayofweek
# df = df.drop(columns=date_cols)  # optional

# ------------------------------------------------------------
# 3) TYPEN & KATEGORIEN (A1, A9)
# ------------------------------------------------------------
cat_cols_guess = list(df.select_dtypes(include=["object"]).columns)
for col in cat_cols_guess:
    df[col] = df[col].str.lower()

# ------------------------------------------------------------
# 4) ZIEL (optional) & FEATURE-SELEKTION
# ------------------------------------------------------------
TARGET_COL = "price" if "price" in df.columns else None

if TARGET_COL is not None:
    y = df[TARGET_COL].astype(float)  # erzwinge numerisch
    X = df.drop(columns=[TARGET_COL])
else:
    y = None
    X = df.copy()

# ------------------------------------------------------------
# 5) SPALTENTYPEN ERKENNEN
# ------------------------------------------------------------
num_cols = list(X.select_dtypes(include=[np.number]).columns)
cat_cols = list(X.select_dtypes(include=["object", "category"]).columns)
dt_cols  = [c for c in X.columns if np.issubdtype(X[c].dtype, np.datetime64)]

print("[INFO] Spalten-Typen:")
print("  numerisch:", num_cols)
print("  kategorisch:", cat_cols)
print("  datetime:", dt_cols, "\n")

# ------------------------------------------------------------
# 6) TRAIN/TEST SPLIT (A10)
# ------------------------------------------------------------
if y is not None:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
else:
    X_train, X_test = train_test_split(X, test_size=0.25, random_state=42)
    y_train = y_test = None

# ------------------------------------------------------------
# 7) BENUTZERWAHL: SKALIERUNG (A4)
# ------------------------------------------------------------
SCALE_KIND = "zscore"  # 'zscore' | 'minmax' | 'robust'

if SCALE_KIND == "zscore":
    scaler = StandardScaler()
elif SCALE_KIND == "minmax":
    scaler = MinMaxScaler(feature_range=(0, 1))
elif SCALE_KIND == "robust":
    scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0))
else:
    raise ValueError("SCALE_KIND muss 'zscore', 'minmax' oder 'robust' sein.")

# ------------------------------------------------------------
# 8) AUSREIẞER-KAPPEN (A5) (Windosirized)
# ------------------------------------------------------------
from sklearn.utils.validation import check_is_fitted

class QuantileClipper(BaseEstimator, TransformerMixin):
    def __init__(self, q_low=0.01, q_high=0.99):
        self.q_low = q_low
        self.q_high = q_high
    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        self.lower_ = X.quantile(self.q_low, axis=0)
        self.upper_ = X.quantile(self.q_high, axis=0)
        return self
    def transform(self, X):
        check_is_fitted(self, ["lower_", "upper_"])
        X = pd.DataFrame(X)
        X = X.clip(lower=self.lower_.values, upper=self.upper_.values, axis=1)
        return X.values

# ------------------------------------------------------------
# 9) PIPELINES (A2, A4, A6, A10)
# ------------------------------------------------------------
num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("clip", QuantileClipper(q_low=0.01, q_high=0.99)),
    ("scaler", scaler),
])

# --- OneHotEncoder: kompatibel zu sklearn>=1.2 (sparse_output) und älter (sparse) ---
def make_ohe():
    try:
        # Neuere sklearn-Versionen
        return OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False)
    except TypeError:
        # Ältere sklearn-Versionen
        return OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse=False)

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", make_ohe()),
])

# Datumswerte droppen (Roh-datetime), abgeleitete Features behalten wir schon oben
drop_cols = dt_cols
X_train_ = X_train.drop(columns=drop_cols) if drop_cols else X_train
X_test_  = X_test.drop(columns=drop_cols)  if drop_cols else X_test

# Spaltenlisten nach Drop aktualisieren
num_cols = [c for c in num_cols if c in X_train_.columns]
cat_cols = [c for c in cat_cols if c in X_train_.columns]

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=True,
)

# ------------------------------------------------------------
# 10) FIT/TRANSFORM
# ------------------------------------------------------------
if y_train is not None:
    _ = preprocess.fit(X_train_, y_train)
else:
    _ = preprocess.fit(X_train_)

X_train_pre = preprocess.transform(X_train_)
X_test_pre  = preprocess.transform(X_test_)

# Feature-Namen
try:
    feat_names = preprocess.get_feature_names_out()
except Exception:
    feat_names = [f"f{i}" for i in range(X_train_pre.shape[1])]

X_train_pre_df = pd.DataFrame(X_train_pre, columns=feat_names, index=X_train_.index)
X_test_pre_df  = pd.DataFrame(X_test_pre,  columns=feat_names, index=X_test_.index)

print("[INFO] Nach Vorbereitung:")
print("  X_train_pre Form:", X_train_pre_df.shape)
print("  X_test_pre  Form:", X_test_pre_df.shape)
print("  Beispielspalten:", list(X_train_pre_df.columns[:min(10, X_train_pre_df.shape[1])]), "\n")

# ------------------------------------------------------------
# 11) DIAGNOSTIK
# ------------------------------------------------------------
missing_after = X_train_pre_df.isna().sum().sum()
print(f"[CHECK] Fehlende Werte nach Pipeline (Train): {missing_after}")

desc = X_train_pre_df.describe().T
print("\n[STAT] Beschreibende Statistik der vorbereiteten Features (Train):")
print(desc.head(12), "\n")

# ------------------------------------------------------------
# 12) EXPORT
# ------------------------------------------------------------
X_train_pre_df.to_csv("prepared_train.csv", index=False)
X_test_pre_df.to_csv("prepared_test.csv", index=False)
print("[INFO] 'prepared_train.csv' und 'prepared_test.csv' gespeichert.")


[INFO] CSV Beispiel-CSV aus String
[INFO] Rohform:
                     date     city  rooms  area_m2     price has_elevator
0              2025-01-03   Berlin    2.0     45.0  245000.0          yes
1              2025-02-10   berlin    NaN     60.0  315000.0           no
2              2024-12-15  München    1.0     34.0       NaN          yes
3              2025-01-20  Hamburg    3.0     85.0  520000.0          yes
4              2025-03-02  München    2.0      NaN  480000.0           no 

[INFO] Duplikate entfernt: 0

[INFO] Spalten-Typen:
  numerisch: ['rooms', 'area_m2', 'date_year', 'date_month', 'date_dow']
  kategorisch: ['city', 'has_elevator']
  datetime: ['date'] 

[INFO] Nach Vorbereitung:
  X_train_pre Form: (4, 11)
  X_test_pre  Form: (2, 11)
  Beispielspalten: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9'] 

[CHECK] Fehlende Werte nach Pipeline (Train): 0

[STAT] Beschreibende Statistik der vorbereiteten Features (Train):
     count          mean       std 